In [2]:
import re
import pandas as pd
import gensim.utils
from gensim.models import Phrases # for collocation detector for word2vec
import nltk
from gensim.parsing import preprocessing
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import KeyedVectors

import sys 
sys.path.append('/Users/jessicahassibi/PycharmProjects/bachelor-thesis/src/data') # replace with your absolute path
# import modules from another folder
import wikipedia_helpers

In [2]:
# path to save wikipedia texts to or load from if file already existing
path = "data/komponisten_texte.json"

# create dictionary with composers as keys and \
# for each key a dictionary with the texts of the wikipedia articles \
# in the languages german, arabic, english, italian, french and spanish
data_dict = wikipedia_helpers.extract_composers_texts(path)
df = pd.read_json(path)
# transpose index and columns of df
df = df.transpose()
df.head()

Texts have already been scraped to: data/komponisten_texte.json


,de_title,de_texts,en_title,en_texts,ar_title,ar_texts,fr_title,fr_texts,it_title,it_texts,es_title,es_texts
1,Abel Ehrlich,"[Ehrlich, aufgewachsen in einer deutschen jüdi...",Abel Ehrlich,"[Erlich was born in 1915 in Cranz, East Prussi...",أبيل إرليخ,[أبيل إرليخ على موقع المكتبة المفتوحة (الإنجلي...,Abel Ehrlich,"[Erlich naît à Cranz, en Prusse-Orientale. En ...",,,,
2,Admiral Horthy,[Horthy stammte aus einer in der Großen Ungari...,Miklós Horthy,[Miklós Horthy de Nagybánya was born at Kender...,ميكلوش هورتي,[ميكلوش هورتي على موقع Encyclopædia Britannica...,Miklós Horthy,"[Après la guerre, la république fédérative pop...",Miklós Horthy,[Horthy nacque in una cittadina nell'attuale p...,Miklós Horthy,[Pál Teleki Gyula Gömbös István Bethlen Primer...
3,Alexander von Zemlinsky,"[Stefan Schmidl: Zemlinsky, Alexander von. In:...",Alexander von Zemlinsky,[Zemlinsky's best-known work is the Lyric Symp...,ألكسندر فون زيميلنسكي,[ألكسندر فون زيميلنسكي على موقع Encyclopædia B...,Alexander von Zemlinsky,[Zemlinsky étudie au conservatoire de Vienne o...,Alexander von Zemlinsky,[Quintetto d'archi 1896 6 quartetti per archi ...,Alexander von Zemlinsky,"[El padre de Alexander, Adolf, provenía de una..."
4,Alexandre Tansman,[Tansman wurde in Łódź als Sohn jüdischer Elte...,Alexandre Tansman,[Tansman was born and raised in the Polish cit...,,,Alexandre Tansman,[Né à Łódź le 11 juin 1897 dans une famille de...,Alexandre Tansman,"[Tansman nacque a Łódź, in Polonia, nel 1897. ...",Alexandre Tansman,[Sus primeros años transcurrieron en su Poloni...
5,Alfons Josef Biron,[],,,,,,,,,,


In [26]:
####################################################################################################################################
####################################################################################################################################
# Methods to get Word Embeddings

In [3]:
####################################################################################################################################
# Using Word2Vec
# preprocessing the texts necessary
de_texts_processed = [[gensim.parsing.preprocess_string(text) for text in row] for row in df.de_texts]
en_texts_processed = [[gensim.parsing.preprocess_string(text) for text in row] for row in df.en_texts]
ar_texts_processed = [[gensim.parsing.preprocess_string(text) for text in row] for row in df.ar_texts]
fr_texts_processed = [[gensim.parsing.preprocess_string(text) for text in row] for row in df.fr_texts]
es_texts_processed = [[gensim.parsing.preprocess_string(text) for text in row] for row in df.es_texts]
it_texts_processed = [[gensim.parsing.preprocess_string(text) for text in row] for row in df.it_texts]
# preprocessed_texts_of_first_article = de_texts_processed[0])

all_sentences_processed = []
for lang in [de_texts_processed, en_texts_processed, ar_texts_processed, fr_texts_processed, es_texts_processed, it_texts_processed]:
    for document in lang:
        for sentence in document:
            all_sentences_processed.append(sentence)

In [30]:
s = "and some white space. Multiple sentences, too. This is nuts!"
gensim.parsing.preprocess_string(s)

['white', 'space', 'multipl', 'sentenc', 'nut']

In [43]:
# not so good
# Train a bigram detector.

bigram_transformer = Phrases(all_sentences_processed)

# Apply the trained MWE detector to a corpus, using the result to train a Word2vec model.
# Gensim implementiert Word2Vec-Algorithmus für das Training der Wortvektoren

model = gensim.models.Word2Vec(bigram_transformer[all_sentences_processed], min_count=2)

In [4]:
model = gensim.models.Word2Vec(all_sentences_processed, min_count=2)

In [5]:
model.train(all_sentences_processed, total_examples=model.corpus_count, epochs=model.epochs)

(1608315, 1834235)

In [45]:
model.build_vocab(all_sentences_processed, progress_per=10)


In [8]:
model.save("word2vec_multilingual.model")
#model = gensim.models.Word2Vec.load("word2vec_multilingual.model")

In [6]:
# get word vector
vector = model.wv['jude']
print(vector)

[ 0.10077599  0.7247989   0.12365346  0.49204093  0.7384333   0.47699213
  0.31028938  0.60216945 -0.31319153 -0.04132529  0.29609028 -0.6989141
  0.10178782  0.31694344  0.21723385 -0.51244706  1.2880293   0.05481082
 -0.15493083 -0.7393516   0.7728396  -0.17083734  0.43076533 -0.10465332
 -0.27478164 -0.03477897 -0.25073907 -0.6565186  -0.17178622 -0.09940004
  0.743909   -0.6097931   0.26710385 -0.8173809   0.13112906  0.58130157
 -0.18727869  0.02610918 -0.25274572 -0.8908919   0.00866548 -0.09663318
 -0.07425451  1.1333903  -0.39793342 -0.00209335 -0.40066776 -0.34980693
  0.12408873  0.44546354  0.07294137  0.44613954 -0.29312778 -0.79084074
 -0.39333105 -0.3842439   0.185867    0.28478017 -0.49091843  0.34022775
 -0.49445912  0.3231665  -0.03719124 -0.06182681 -0.9792509   0.41807947
 -0.23925251  0.59832585 -0.7666748   0.06306246  0.37409124  0.11655071
  0.6374261   0.4862683   0.34604225 -0.3981631  -0.02112775  0.6907374
 -0.4848311  -0.5291851  -0.8425719  -0.5126301  -0.1

In [17]:
#variant 2: save model with keyed vectors
# no need for that ?!?! 
word_vectors = model.wv
#word_vectors.save('vectors.kv')
reloaded_word_vectors = KeyedVectors.load('vectors.kv')
word2vec.get_word_in_static_embbedings(word_vectors, all_sentences_processed, save_path="word2vec_kv_models/word2vec_kv_model")

vocab ['der', 'und', 'die', 'von', 'de', 'music', 'für', 'ein', 'mit', 'piano', 'isbn', 'da', 'musik', 'berlin', 'den', 'pour', 'al', 'nach', 'sein', 'war', 'dem', 'wurd', 'orchestra', 'del', 'compos', 'work', 'le', 'musiqu', 'auf', 'que', 'bei', 'wien', 'new', 'est', 'symphoni', 'una', 'son', 'lo', 'sich', 'bi', 'paul', 'au', 'auch', 'nel', 'dan', 'york', 'composit', 'werk', 'über', 'par', 'orchestr', 'film', 'un', 'record', 'klavier', 'opera', 'univers', 'على', 'avec', 'seiner', 'della', 'han', 'para', 'vienna', 'oper', 'concerto', 'verlag', 'schönberg', 'portail', 'por', 'jahr', 'franz', 'band', 'bartók', 'song', 'archiv', 'perform', 'studi', 'zur', 'art', 'einer', 'max', 'deutsch', 'vom', 'text', 'che', 'richard', 'hrsg', 'pari', 'german', 'deutschen', 'intern', 'nazi', 'zeit', 'zum', 'sur', 'violin', 'internet', 'ernst', 'dell', 'sie', 'score', 'se', 'orchest', 'komponisten', 'suit', 'musica', 'onlin', 'librari', 'georg', 'lexikon', 'ist', 'œuvr', 'edit', 'unter', 'موقع', 'london'

Check which word are in the static embeddings:   0%|                                                                                               | 0/3654 [00:00<?, ?it/s]

['ehrlich', 'aufgewachsen', 'einer', 'deutschen', 'jüdischen', 'famili', 'floh', 'nach', 'dem', 'abitur', 'königsberg', 'nach', 'jugoslawien', 'studiert', 'der', 'musikakademi', 'zagreb', 'komposit', 'und', 'violin', 'bei', 'vaclav', 'huml', 'emigriert', 'über', 'albanien', 'da', 'damalig', 'palästina', 'studiert', 'konservatorium', 'jerusalem', 'bei', 'emil', 'hauser', 'tzvi', 'rothenberg', 'und', 'solomon', 'rosowski', 'selbst', 'unterrichtet', 'danach', 'verschiedenen', 'musikakademien', 'jerusalem', 'und', 'tel', 'aviv', 'darunt', 'auch', 'von', 'bi', 'der', 'kibbuzim', 'hochschul', 'oranim', 'wurd', 'professor', 'der', 'damaligen', 'rubin', 'akademi', 'tel', 'aviv', 'bi', 'auch', 'der', 'bar', 'ilan', 'universität', 'lehrt', 'komponiert', 'werk', 'darunt', 'opern', 'oratorien', 'sowi', 'chor', 'und', 'solovokalmusik', 'orchesterwerk', 'ballett', 'schauspiel', 'kammermusik', 'und', 'elektroakustisch', 'musik', 'berühmt', 'wurd', 'sein', 'komposit', 'bashrav', 'für', 'soloviolin', '

AttributeError: 'list' object has no attribute 'lower'

In [21]:
# find words used in similar context
model.wv.most_similar("composer")

KeyError: "Key 'composer' not present in vocabulary"

In [19]:
model.wv.distance("schönberg", "webern")

0.26980578899383545

In [86]:
model.wv.similarity(w1="komponist", w2="nazi") 

0.6665484

In [87]:
model.wv.doesnt_match(['pianist', 'violinist', 'nazi'])

'nazi'

In [88]:
model.wv.most_similar(positive=["komponist", "jude"], negative=["schönberg"])

[('eher', 0.9178099632263184),
 ('französisch', 0.9147974252700806),
 ('politischen', 0.9130923748016357),
 ('dienst', 0.9129126667976379),
 ('geworden', 0.9105923771858215),
 ('krieg', 0.9105893969535828),
 ('damaligen', 0.9100316166877747),
 ('staatsbürgerschaft', 0.9097257256507874),
 ('verurteilt', 0.9095054268836975),
 ('bibliotheken', 0.9088475108146667)]